# CCTV Analysis
This notebook demonstrates the capabilities of our CCTV analysis system for multi-camera person tracking and analysis.


In [1]:
# Import required libraries
import numpy as np
import torch
import cv2
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Import our modules
from cctv_analysis.detector import PersonDetector
from cctv_analysis.tracker import PersonTracker
from cctv_analysis.reid import PersonReID
from cctv_analysis.matcher import CameraPersonMatcher
from cctv_analysis.demographics import DemographicAnalyzer
from cctv_analysis.utils.visualization import Visualizer
from cctv_analysis.utils.metrics import TrackingMetrics, MultiCameraMetrics, DemographicMetrics
from cctv_analysis.utils.config import ConfigLoader

## 1. System Setup
First, let's initialize our models and configuration. We're using:
- YOLOv8x6 for person detection
- OSNet for person re-identification

In [ ]:
# Load configuration
config = ConfigLoader("configs/models.yaml")
model_config = config.get_model_config()
tracking_config = config.get_tracking_config()

In [ ]:
# Initialize detector with GPU acceleration
detector = PersonDetector(
    model_path="models/detector/yolov8x6.pt",
    conf_thresh=0.5
)
print(f"Using device: {detector.device}")

In [ ]:
# Initialize tracker with our configured settings
tracker = PersonTracker(
    reid_model=reid_model,
    max_age=tracking_config.max_age,
    min_hits=tracking_config.min_hits
)

In [ ]:
# Initialize visualizer for displaying results
visualizer = Visualizer(num_colors=100)

## 2. Single Camera Tracking Demo
Let's start by demonstrating how our system tracks people in a single camera view. We'll process a video 
and display the results frame by frame, showing how the tracker maintains consistent identities even when 
people move around or are temporarily occluded.

In [ ]:
def process_single_camera(video_path: str, max_frames: int = 100, display_interval: int = 5):
    """
    Process a single camera video feed and display tracking results.

    Args:
        video_path: Path to the input video file
        max_frames: Maximum number of frames to process
        display_interval: Show every nth frame
    """
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    processing_metrics = {
        'total_detections': 0,
        'average_confidence': 0,
        'unique_tracks': set()
    }

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to RGB for processing
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Step 1: Detect people in the frame
        detections = detector.detect(frame_rgb)
        processing_metrics['total_detections'] += len(detections)
        if detections:
            processing_metrics['average_confidence'] += sum(
                conf for _, conf in detections)

        # Step 2: Update tracking with new detections
        tracks = tracker.update(frame_rgb, detections)
        for track in tracks:
            processing_metrics['unique_tracks'].add(track.track_id)

        # Display results at specified intervals
        if frame_count % display_interval == 0:
            vis_frame = visualizer.draw_tracks(
                frame_rgb,
                [(t.track_id, t.bbox) for t in tracks]
            )

            # Add frame information overlay
            info_text = [
                f"Frame: {frame_count}",
                f"Active Tracks: {len(tracks)}",
                f"Total Unique IDs: {len(processing_metrics['unique_tracks'])}"
            ]

            # Display frame with tracking results
            plt.figure(figsize=(15, 8))
            plt.imshow(vis_frame)
            plt.title("Person Tracking Results", pad=20)

            # Add metrics text
            for i, text in enumerate(info_text):
                plt.text(10, 20 + i * 20, text, color='white',
                         backgroundcolor='black', fontsize=10)

            plt.axis('off')
            plt.show()
            plt.close()

        frame_count += 1

    # Calculate and display final metrics
    if processing_metrics['total_detections'] > 0:
        avg_conf = processing_metrics['average_confidence'] / \
            processing_metrics['total_detections']
    else:
        avg_conf = 0

    print("\nProcessing Summary:")
    print(f"Total frames processed: {frame_count}")
    print(
        f"Average detections per frame: {processing_metrics['total_detections']/frame_count:.2f}")
    print(f"Average detection confidence: {avg_conf:.3f}")
    print(f"Total unique tracks: {len(processing_metrics['unique_tracks'])}")

    cap.release()

## 3. Multi-Camera Tracking Demo
Now let's demonstrate the system's ability to track people across multiple camera views. This is more 
challenging as we need to maintain consistent identities even when people disappear from one camera and 
reappear in another.

In [ ]:
# Initialize our cross-camera matcher
matcher = CameraPersonMatcher(
    reid_model=reid_model,
    matching_threshold=0.7
)

In [ ]:
def process_multiple_cameras(video_paths: dict, max_frames: int = 100, display_interval: int = 5):
    """
    Process multiple synchronized video feeds and demonstrate cross-camera tracking.

    Args:
        video_paths: Dictionary mapping camera IDs to video file paths
        max_frames: Maximum number of frames to process
        display_interval: Show every nth frame
    """
    # Open all video captures
    captures = {
        cam_id: cv2.VideoCapture(path)
        for cam_id, path in video_paths.items()
    }

    # Initialize separate trackers for each camera
    trackers = {
        cam_id: PersonTracker(reid_model=reid_model)
        for cam_id in video_paths.keys()
    }

    # Initialize metrics tracking
    cross_camera_metrics = {
        'global_ids': set(),
        'transitions': 0,
        'camera_appearances': defaultdict(int)
    }

    frame_count = 0
    while frame_count < max_frames:
        current_time = datetime.now()
        frames = {}
        detections = {}
        tracks = {}

        # Process each camera feed
        for cam_id, cap in captures.items():
            ret, frame = cap.read()
            if not ret:
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames[cam_id] = frame_rgb

            # Detect and track people in this camera
            dets = detector.detect(frame_rgb)
            detections[cam_id] = dets
            tracks[cam_id] = trackers[cam_id].update(frame_rgb, dets)

            # Update global matches
            for track in tracks[cam_id]:
                # Get global ID for this track
                global_id = matcher.update(
                    camera_id=cam_id,
                    track_id=track.track_id,
                    feature=track.feature,
                    timestamp=current_time
                )

                # Update metrics
                cross_camera_metrics['global_ids'].add(global_id)
                cross_camera_metrics['camera_appearances'][cam_id] += 1

        # Display results at specified intervals
        if frame_count % display_interval == 0:
            # Get visualization with global IDs
            vis_frames = visualizer.draw_multicamera_matches(
                frames=frames,
                matches={cam_id: [(t.track_id, t.bbox) for t in cam_tracks]
                         for cam_id, cam_tracks in tracks.items()},
                global_ids={cam_id: {t.track_id: matcher.get_global_id(cam_id, t.track_id)
                                     for t in cam_tracks}
                            for cam_id, cam_tracks in tracks.items()}
            )

            # Create multi-camera display
            plt.figure(figsize=(20, 5 * len(captures)))
            for idx, (cam_id, frame) in enumerate(vis_frames.items(), 1):
                plt.subplot(len(captures), 1, idx)
                plt.imshow(frame)
                plt.title(
                    f"Camera {cam_id} - Active Tracks: {len(tracks[cam_id])}")
                plt.axis('off')

            plt.tight_layout()
            plt.show()
            plt.close()

        frame_count += 1

    # Display final cross-camera statistics
    print("\nCross-Camera Tracking Summary:")
    print(
        f"Total unique people tracked: {len(cross_camera_metrics['global_ids'])}")
    print("\nCamera Appearances:")
    for cam_id, count in cross_camera_metrics['camera_appearances'].items():
        print(f"Camera {cam_id}: {count} total detections")

    # Get popular transition paths
    popular_paths = matcher.get_popular_paths(top_k=5)
    print("\nMost Common Camera Transitions:")
    for (cam1, cam2), count in popular_paths:
        print(f"Camera {cam1} → Camera {cam2}: {count} transitions")

    # Release all video captures
    for cap in captures.values():
        cap.release()

## 4. Demographic Analysis Demo
Let's demonstrate how our system can analyze the demographics of tracked individuals over time.

In [ ]:
# Initialize our demographic analyzer
demographic_analyzer = DemographicAnalyzer(
    gender_model_path="models/demographics/gender_model.pth",
    age_model_path="models/demographics/age_model.pth"
)

In [ ]:
def analyze_demographics_over_time(video_path: str, max_frames: int = 100,
                                   analysis_interval: int = 10):
    """
    Demonstrate demographic analysis capabilities on a video stream.

    Args:
        video_path: Path to input video
        max_frames: Maximum frames to process
        analysis_interval: Perform demographic analysis every n frames
    """
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    # Initialize metrics collectors
    demographic_metrics = DemographicMetrics()
    tracking_metrics = TrackingMetrics()

    # Store historical data for visualization
    temporal_data = {
        'timestamps': [],
        'gender_ratios': [],
        'age_distributions': []
    }

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = datetime.now()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect and track people
        detections = detector.detect(frame_rgb)
        tracks = tracker.update(frame_rgb, detections)

        # Perform demographic analysis at intervals
        if frame_count % analysis_interval == 0:
            frame_demographics = {}

            # Analyze each tracked person
            for track in tracks:
                x1, y1, x2, y2 = track.bbox
                person_patch = frame_rgb[y1:y2, x1:x2]

                demo_info = demographic_analyzer.analyze_person(person_patch)
                if demo_info:
                    frame_demographics[track.track_id] = demo_info
                    demographic_metrics.update(
                        timestamp=current_time,
                        gender=demo_info.gender,
                        age_group=demo_info.age_group
                    )

            # Update temporal data
            distribution = demographic_metrics.get_distribution()
            temporal_data['timestamps'].append(current_time)
            temporal_data['gender_ratios'].append(distribution['gender'])
            temporal_data['age_distributions'].append(distribution['age'])

            # Visualize current frame with demographic information
            vis_frame = visualizer.draw_tracks_with_demographics(
                frame_rgb,
                [(t.track_id, t.bbox) for t in tracks],
                frame_demographics
            )

            # Display results
            plt.figure(figsize=(15, 10))

            # Main frame with tracking and demographics
            plt.subplot(2, 1, 1)
            plt.imshow(vis_frame)
            plt.title(f"Frame {frame_count} - Demographic Analysis")
            plt.axis('off')

            # Current demographic distribution
            plt.subplot(2, 2, 3)
            gender_dist = distribution['gender']
            plt.bar(gender_dist.keys(), gender_dist.values())
            plt.title("Gender Distribution")
            plt.ylabel("Proportion")

            plt.subplot(2, 2, 4)
            age_dist = distribution['age']
            plt.bar(age_dist.keys(), age_dist.values())
            plt.title("Age Distribution")
            plt.xticks(rotation=45)

            plt.tight_layout()
            plt.show()
            plt.close()

        frame_count += 1

    # Display final analysis
    print("\nDemographic Analysis Summary:")
    final_dist = demographic_metrics.get_distribution()

    print("\nOverall Gender Distribution:")
    for gender, ratio in final_dist['gender'].items():
        print(f"{gender}: {ratio:.1%}")

    print("\nOverall Age Distribution:")
    for age_group, ratio in final_dist['age'].items():
        print(f"{age_group}: {ratio:.1%}")

    # Get temporal patterns
    hourly_patterns = demographic_metrics.get_hourly_patterns()
    print("\nPeak Activity Periods:")
    for hour, stats in hourly_patterns.items():
        if stats['total_count'] > 0:
            print(f"\nHour {hour:02d}:00:")
            print(f"Total people: {stats['total_count']}")
            print("Gender distribution:",
                  {k: f"{v:.1%}" for k, v in stats['gender_distribution'].items()})

    cap.release()

## 5. Analysis of Traffic Patterns and Flow

Now that we've covered the basics of tracking and demographics, let's explore how we can analyze 
the movement patterns of people through our camera network. Understanding traffic flow helps in 
optimizing space usage and identifying potential bottlenecks.

In [ ]:
def analyze_traffic_patterns(video_paths: dict, duration_minutes: float = 5.0):
    """
    Analyze traffic patterns across multiple cameras over a specified duration.

    This function helps us understand:
    - How people move between different areas
    - Which paths are most commonly taken
    - Peak traffic times and locations
    - Potential congestion points
    """
    # Convert duration to frames (assuming 30 fps)
    max_frames = int(duration_minutes * 60 * 30)

    # Initialize our flow tracking system
    flow_metrics = {
        'transitions': defaultdict(int),  # Camera-to-camera transitions
        'dwell_times': defaultdict(list),  # Time spent in each camera view
        # Traffic by location and time
        'traffic_density': defaultdict(lambda: defaultdict(int)),
        'path_sequences': []  # Complete paths taken by individuals
    }

    # Process videos
    captures = {cam_id: cv2.VideoCapture(path)
                for cam_id, path in video_paths.items()}

    try:
        for frame_idx in tqdm(range(max_frames), desc="Analyzing traffic patterns"):
            frames = {}
            current_time = datetime.now()

            # Process each camera
            for cam_id, cap in captures.items():
                ret, frame = cap.read()
                if not ret:
                    continue

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Detect and track people
                detections = detector.detect(frame_rgb)
                tracks = tracker.update(frame_rgb, detections)

                # Update global tracking
                for track in tracks:
                    # Get global ID for consistent tracking across cameras
                    global_id = matcher.update(
                        camera_id=cam_id,
                        track_id=track.track_id,
                        feature=track.feature,
                        timestamp=current_time
                    )

                    # Record movements and patterns
                    _update_flow_metrics(
                        flow_metrics, global_id, cam_id, track, current_time)

            # Periodically visualize the flow patterns
            if frame_idx % 30 == 0:  # Every second
                _visualize_flow_patterns(flow_metrics, frames)

    finally:
        # Clean up
        for cap in captures.values():
            cap.release()

    return flow_metrics

In [ ]:
def _update_flow_metrics(metrics, global_id, camera_id, track, timestamp):
    """
    Update traffic flow metrics with new tracking information.
    """
    # Get previous location of this person
    prev_camera = matcher.get_previous_camera(global_id, camera_id)

    if prev_camera is not None and prev_camera != camera_id:
        # Record camera transition
        transition_key = (prev_camera, camera_id)
        metrics['transitions'][transition_key] += 1

        # Update path sequence
        metrics['path_sequences'].append(
            (global_id, prev_camera, camera_id, timestamp))

    # Record dwell time in current camera
    metrics['dwell_times'][camera_id].append({
        'global_id': global_id,
        'timestamp': timestamp,
        'position': track.bbox  # This helps analyze specific areas within the camera view
    })

    # Update traffic density (divide frame into 5x5 grid)
    x1, y1, x2, y2 = track.bbox
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2
    grid_x = center_x // (frame_width // 5)
    grid_y = center_y // (frame_height // 5)
    metrics['traffic_density'][camera_id][(grid_x, grid_y)] += 1

In [ ]:
def _visualize_flow_patterns(metrics, frames):
    """
    Create visualizations of current traffic patterns.
    """
    plt.figure(figsize=(20, 10))

    # 1. Traffic Flow Graph
    plt.subplot(2, 2, 1)
    G = nx.DiGraph()

    # Add edges with weights based on transition counts
    for (cam1, cam2), count in metrics['transitions'].items():
        G.add_edge(f"Cam {cam1}", f"Cam {cam2}", weight=count)

    # Draw the graph
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color='lightblue',
            node_size=1000, font_size=10, font_weight='bold')

    edge_weights = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_weights)

    plt.title("Traffic Flow Between Cameras")

    # 2. Heatmap of Traffic Density
    plt.subplot(2, 2, 2)
    for cam_id, density in metrics['traffic_density'].items():
        density_matrix = np.zeros((5, 5))
        for (x, y), count in density.items():
            density_matrix[y, x] = count

        plt.imshow(density_matrix, cmap='YlOrRd')
        plt.colorbar(label='Traffic Count')
        plt.title(f"Traffic Density Heatmap - Camera {cam_id}")

        # Add grid lines
        for i in range(6):
            plt.axhline(y=i-0.5, color='black', linewidth=1)
            plt.axvline(x=i-0.5, color='black', linewidth=1)

    plt.tight_layout()
    plt.show()

## 6. Behavior Analysis

Let's add functionality to analyze various behavioral patterns and interactions.
This can help identify unusual activities or understand how people use the space.

In [ ]:
class BehaviorAnalyzer:
    """
    Analyzes behavioral patterns in surveillance footage.
    """

    def __init__(self):
        self.interaction_threshold = 100  # pixels
        self.stationary_threshold = 50    # pixels
        self.time_window = 30             # frames

        # Store historical data
        self.position_history = defaultdict(
            lambda: deque(maxlen=self.time_window))
        self.interaction_history = defaultdict(list)
        self.behavior_patterns = defaultdict(list)

    def analyze_frame(self, tracks, timestamp):
        """
        Analyze behaviors in the current frame.
        """
        # Update position history
        current_positions = {}
        for track in tracks:
            x1, y1, x2, y2 = track.bbox
            center = ((x1 + x2) // 2, (y1 + y2) // 2)
            current_positions[track.track_id] = center
            self.position_history[track.track_id].append((center, timestamp))

        # Analyze behaviors
        behaviors = {
            'stationary': self._detect_stationary_people(current_positions),
            'interactions': self._detect_interactions(current_positions),
            'rapid_movement': self._detect_rapid_movement(current_positions),
            'groups': self._detect_groups(current_positions)
        }

        return behaviors

    def _detect_stationary_people(self, current_positions):
        """
        Identify people who have remained relatively still.
        """
        stationary = []
        for track_id, history in self.position_history.items():
            if len(history) < self.time_window:
                continue

            # Calculate total movement
            positions = [pos for pos, _ in history]
            total_movement = sum(np.linalg.norm(np.array(p2) - np.array(p1))
                                 for p1, p2 in zip(positions[:-1], positions[1:]))

            if total_movement < self.stationary_threshold:
                stationary.append(track_id)

        return stationary

    def _detect_interactions(self, current_positions):
        """
        Detect potential interactions between people based on proximity.
        """
        interactions = []
        positions = list(current_positions.items())

        for i in range(len(positions)):
            for j in range(i + 1, len(positions)):
                id1, pos1 = positions[i]
                id2, pos2 = positions[j]

                distance = np.linalg.norm(np.array(pos1) - np.array(pos2))
                if distance < self.interaction_threshold:
                    interactions.append((id1, id2))

        return interactions

    def _detect_rapid_movement(self, current_positions):
        """
        Identify people moving unusually quickly.
        """
        rapid_movers = []
        for track_id, history in self.position_history.items():
            if len(history) < 2:
                continue

            # Calculate recent velocity
            recent_positions = [pos for pos, _ in history[-2:]]
            velocity = np.linalg.norm(np.array(recent_positions[1]) -
                                      np.array(recent_positions[0]))

            if velocity > self.stationary_threshold * 2:
                rapid_movers.append(track_id)

        return rapid_movers

    def _detect_groups(self, current_positions):
        """
        Identify groups of people using clustering.
        """
        if len(current_positions) < 2:
            return []

        # Convert positions to array for clustering
        positions = np.array(list(current_positions.values()))
        track_ids = list(current_positions.keys())

        # Use DBSCAN for clustering
        clustering = DBSCAN(eps=self.interaction_threshold,
                            min_samples=2).fit(positions)

        # Organize results into groups
        groups = defaultdict(list)
        for track_id, label in zip(track_ids, clustering.labels_):
            if label >= 0:  # Ignore noise points (-1)
                groups[label].append(track_id)

        return list(groups.values())

## 7. System Performance Analysis

Let's analyze the performance of our tracking system and identify areas for improvement.

In [ ]:
def analyze_system_performance(video_paths: dict, duration_seconds: int = 30):
    """
    Analyze and report on system performance metrics.
    """
    performance_metrics = {
        'processing_times': defaultdict(list),
        'detection_confidence': defaultdict(list),
        'tracking_consistency': defaultdict(list),
        'reid_accuracy': defaultdict(list),
        'gpu_utilization': [],
        'memory_usage': []
    }

    start_time = time.time()
    frame_count = 0
    max_frames = duration_seconds * 30  # Assuming 30 fps

    # Process videos
    captures = {cam_id: cv2.VideoCapture(path)
                for cam_id, path in video_paths.items()}

    try:
        while frame_count < max_frames:
            frame_start = time.time()

            for cam_id, cap in captures.items():
                ret, frame = cap.read()
                if not ret:
                    continue

                # Measure detection time
                det_start = time.time()
                detections = detector.detect(frame)
                det_time = time.time() - det_start
                performance_metrics['processing_times']['detection'].append(
                    det_time)

                # Record detection confidences
                if detections:
                    confidences = [conf for _, conf in detections]
                    performance_metrics['detection_confidence'][cam_id].extend(
                        confidences)

                # Measure tracking time
                track_start = time.time()
                tracks = tracker.update(frame, detections)
                track_time = time.time() - track_start
                performance_metrics['processing_times']['tracking'].append(
                    track_time)

                # Measure ReID time
                reid_start = time.time()
                for track in tracks:
                    _ = matcher.update(cam_id, track.track_id,
                                       track.feature, datetime.now())
                reid_time = time.time() - reid_start
                performance_metrics['processing_times']['reid'].append(
                    reid_time)

            # Record resource usage
            if torch.cuda.is_available():
                performance_metrics['gpu_utilization'].append(
                    torch.cuda.memory_allocated() / torch.cuda.max_memory_allocated()
                )

            performance_metrics['memory_usage'].append(
                psutil.Process().memory_info().rss / (1024 * 1024)  # MB
            )

            frame_count += 1

    finally:
        for cap in captures.values():
            cap.release()

    # Calculate and display performance statistics
    total_time = time.time() - start_time
    fps = frame_count / total_time

    print("\nSystem Performance Analysis")
    print("=" * 50)

    print(f"\nOverall Performance:")
    print(f"Average FPS: {fps:.2f}")
    print(f"Total processing time: {total_time:.2f} seconds")

    print("\nProcessing Times (ms):")
    for component, times in performance_metrics['processing_times'].items():
        avg_time = np.mean(times) * 1000
        std_time = np.std(times) * 1000
        print(f"{component:>10}: {avg_time:>8.2f} ± {std_time:>6.2f}")

    print("\nDetection Quality:")
    for cam_id, confidences in performance_metrics['detection_confidence'].items():
        print(f"Camera {cam_id}:")
        print(f"  Average confidence: {np.mean(confidences):.3f}")
        print(f"  Min confidence: {np.min(confidences):.3f}")
        print(f"  Max confidence: {np.max(confidences):.3f}")

    if performance_metrics['gpu_utilization']:
        print("\nResource Usage:")
        print(
            f"Average GPU utilization: {np.mean(performance_metrics['gpu_utilization']):.1%}")
        print(
            f"Peak GPU utilization: {np.max(performance_metrics['gpu_utilization']):.1%}")
        print(
            f"Average memory usage: {np.mean(performance_metrics['memory_usage']):.1f} MB")
        print(
            f"Peak memory usage: {np.max(performance_metrics['memory_usage']):.1f} MB")

    # Visualize performance metrics
    plt.figure(figsize=(15, 10))

    # Processing times
    plt.subplot(2, 2, 1)
    times_df = pd.DataFrame(performance_metrics['processing_times'])
    times_df.boxplot()
    plt.title('Processing Times Distribution')
    plt.ylabel('Time (seconds)')

    # Detection confidence histogram
    plt.subplot(2, 2, 2)
    for cam_id, confidences in performance_metrics['detection_confidence'].items():
        plt.hist(confidences, alpha=0.5, label=f'Camera {cam_id}')
    plt.title('Detection Confidence Distribution')
    plt.xlabel('Confidence Score')
    plt.ylabel('Frequency')
    plt.legend()

    # GPU utilization over time
    plt.subplot(2, 2, 3)
    plt.plot(performance_metrics['gpu_utilization'])
    plt.title('GPU Utilization Over Time')
    plt.xlabel('Frame')
    plt.ylabel('Utilization %')

    # Memory usage over time
    plt.subplot(2, 2, 4)
    plt.plot(performance_metrics['memory_usage'])
    plt.title('Memory Usage Over Time')
    plt.xlabel('Frame')
    plt.ylabel('Memory Usage (MB)')

    plt.tight_layout()
    plt.show()

    return performance_metrics

## 8. Best Practices and Optimization Guidelines

Now that we've explored the system's capabilities and analyzed its performance, let's discuss
best practices for deploying and optimizing the CCTV analysis system.

In [ ]:
def demonstrate_optimization_techniques():
    """
    Demonstrate various optimization techniques and their impact on system performance.
    """
    print("Optimization Techniques for CCTV Analysis System")
    print("=" * 50)

    print("\n1. Camera Setup Optimization:")
    print("   - Optimal camera placement height: 2.5-3.5 meters")
    print("   - Recommended camera angle: 15-30 degrees downward")
    print("   - Minimum resolution: 1080p for accurate detection")
    print("   - Frame rate: 15-30 fps depending on scenario")

    print("\n2. Detection Optimization:")
    print("   - Batch processing for multiple cameras")
    print("   - Adaptive confidence thresholds")
    print("   - Region of Interest (ROI) masking")

    print("\n3. Tracking Optimization:")
    print("   - Motion prediction for occlusion handling")
    print("   - Feature caching for frequent re-identification")
    print("   - Track pruning for memory efficiency")

    print("\n4. Resource Management:")
    print("   - GPU memory optimization")
    print("   - Batch size tuning")
    print("   - Pipeline parallelization")

    # Example of ROI masking impact
    def demonstrate_roi_masking():
        """Demonstrate the impact of ROI masking on processing time."""
        frame = cv2.imread("sample_frame.jpg")
        if frame is None:
            return

        height, width = frame.shape[:2]

        # Create ROI mask (example: focus on central area)
        mask = np.zeros((height, width), dtype=np.uint8)
        roi_margin = int(width * 0.2)  # 20% margin from edges
        mask[roi_margin:-roi_margin, roi_margin:-roi_margin] = 255

        # Compare processing times
        times_no_roi = []
        times_with_roi = []

        for _ in range(10):
            # Without ROI
            start = time.time()
            detector.detect(frame)
            times_no_roi.append(time.time() - start)

            # With ROI
            start = time.time()
            masked_frame = cv2.bitwise_and(frame, frame, mask=mask)
            detector.detect(masked_frame)
            times_with_roi.append(time.time() - start)

        print("\nROI Masking Performance Impact:")
        print(
            f"Without ROI: {np.mean(times_no_roi)*1000:.2f}ms ± {np.std(times_no_roi)*1000:.2f}ms")
        print(
            f"With ROI: {np.mean(times_with_roi)*1000:.2f}ms ± {np.std(times_with_roi)*1000:.2f}ms")
        print(
            f"Speed improvement: {(1 - np.mean(times_with_roi)/np.mean(times_no_roi))*100:.1f}%")

## 9. Additional Analysis Tools

Let's provide some additional tools for analyzing the surveillance data and extracting insights.

In [ ]:
def analyze_crowd_dynamics(tracks: List[Track], frame_size: Tuple[int, int]):
    """
    Analyze crowd dynamics and movement patterns.
    """
    frame_height, frame_width = frame_size

    # Create grid for density analysis
    grid_size = 32  # pixels per grid cell
    density_map = np.zeros(
        (frame_height // grid_size, frame_width // grid_size))

    # Analyze track positions
    for track in tracks:
        x1, y1, x2, y2 = track.bbox
        center_x = (x1 + x2) // 2
        center_y = (y1 + y2) // 2

        # Update density map
        grid_x = center_x // grid_size
        grid_y = center_y // grid_size
        if 0 <= grid_x < density_map.shape[1] and 0 <= grid_y < density_map.shape[0]:
            density_map[grid_y, grid_x] += 1

    return {
        'density_map': density_map,
        'hotspots': np.where(density_map > np.mean(density_map) + np.std(density_map)),
        'average_density': np.mean(density_map),
        'peak_density': np.max(density_map)
    }

In [ ]:
def generate_analysis_report(video_paths: dict, analysis_duration: int = 300):
    """
    Generate a comprehensive analysis report including all metrics and insights.
    """
    print("Generating Comprehensive Analysis Report")
    print("=" * 50)

    # Initialize analysis components
    behavior_analyzer = BehaviorAnalyzer()
    all_metrics = defaultdict(list)

    # Process videos
    for cam_id, video_path in video_paths.items():
        print(f"\nAnalyzing Camera {cam_id}...")
        cap = cv2.VideoCapture(video_path)

        frame_count = 0
        while cap.isOpened() and frame_count < analysis_duration:
            ret, frame = cap.read()
            if not ret:
                break

            # Process frame
            detections = detector.detect(frame)
            tracks = tracker.update(frame, detections)

            # Analyze behaviors
            behaviors = behavior_analyzer.analyze_frame(tracks, datetime.now())

            # Analyze crowd dynamics
            crowd_analysis = analyze_crowd_dynamics(tracks, frame.shape[:2])

            # Update metrics
            all_metrics['detections'].append(len(detections))
            all_metrics['tracks'].append(len(tracks))
            all_metrics['crowd_density'].append(
                crowd_analysis['average_density'])
            all_metrics['behaviors'].append(behaviors)

            frame_count += 1

        cap.release()

    # Generate report sections
    def _generate_crowd_analysis():
        avg_density = np.mean(all_metrics['crowd_density'])
        peak_density = np.max(all_metrics['crowd_density'])

        print("\nCrowd Analysis:")
        print(f"Average crowd density: {avg_density:.2f} people per cell")
        print(f"Peak crowd density: {peak_density:.2f} people per cell")

        # Visualize density over time
        plt.figure(figsize=(10, 5))
        plt.plot(all_metrics['crowd_density'])
        plt.title('Crowd Density Over Time')
        plt.xlabel('Frame')
        plt.ylabel('Average Density')
        plt.show()

    def _generate_behavior_analysis():
        print("\nBehavior Analysis:")

        # Analyze interaction patterns
        all_interactions = []
        for behaviors in all_metrics['behaviors']:
            all_interactions.extend(behaviors['interactions'])

        print(f"Total interactions detected: {len(all_interactions)}")

        # Analyze group formations
        group_sizes = []
        for behaviors in all_metrics['behaviors']:
            group_sizes.extend([len(group) for group in behaviors['groups']])

        if group_sizes:
            print(f"Average group size: {np.mean(group_sizes):.1f} people")
            print(f"Largest group detected: {np.max(group_sizes)} people")

    # Generate final report
    print("\nFinal Analysis Report")
    print("=" * 50)

    print("\nOverall Statistics:")
    print(f"Total frames analyzed: {len(all_metrics['detections'])}")
    print(
        f"Average detections per frame: {np.mean(all_metrics['detections']):.1f}")
    print(f"Average tracks per frame: {np.mean(all_metrics['tracks']):.1f}")

    _generate_crowd_analysis()
    _generate_behavior_analysis()

    return all_metrics

## 10. Conclusion and Next Steps

Our CCTV analysis system provides a comprehensive solution for multi-camera person tracking
and analysis. Here are the key takeaways and recommendations for deployment:

1. System Capabilities:
   - Real-time person detection and tracking
   - Cross-camera identity matching
   - Demographic analysis
   - Behavior and interaction analysis
   - Traffic pattern analysis

2. Performance Considerations:
   - GPU acceleration is crucial for real-time processing
   - Batch processing can improve throughput
   - ROI masking can reduce computational load
   - Memory management is important for long-term stability

3. Best Practices:
   - Regular system calibration
   - Proper camera placement and configuration
   - Regular model updates and fine-tuning
   - Monitoring of system resources

4. Future Improvements:
   - Integration of anomaly detection
   - Enhanced behavior analysis
   - Real-time alerting system
   - Custom model training for specific scenarios

To get started with your own deployment:
1. Configure camera settings according to the optimization guidelines
2. Set up the required dependencies using the provided requirements.txt
3. Initialize the system components with appropriate parameters
4. Monitor and adjust settings based on performance metrics
5. Regularly validate the system's accuracy and performance


In [ ]:
# Example usage of the complete system
if __name__ == "__main__":
    # Define video sources
    video_paths = {
        1: "data/videos/camera1.mp4",
        2: "data/videos/camera2.mp4",
        3: "data/videos/camera3.mp4"
    }

    # Analyze system performance
    print("Analyzing system performance...")
    performance_metrics = analyze_system_performance(video_paths)

    # Generate comprehensive analysis report
    print("\nGenerating analysis report...")
    analysis_metrics = generate_analysis_report(video_paths)

    print("\nAnalysis complete! Check the visualizations and metrics above for insights.")